<a href="https://colab.research.google.com/github/fatasfaps/duelingddqn-for-sepsis/blob/main/bigquey_initial_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="font-size:32px; font-weight:600; margin-bottom:12px;">
<b>DATA EXTRACTION FROM MIMIC-III</b>
</h1>

<p style="font-size:14px;">
This notebook contains SQL queries required to recreate dataset for the publication:
<a href="https://www.nature.com/articles/s41591-018-0213-5">
https://www.nature.com/articles/s41591-018-0213-5
</a>
</p>

<p style="font-size:14px;">Recreating the dataset is a 3-step process:</p>

<ul style="font-size:14px;">
  <li>Initial extraction from MIMIC-III <b>BigQuery</b>, sub-tables exported as csv files</li>
  <li>Data imported in Google Colaboratory for:
    <ul>
      <li>Sepsis-3 cohort definition (suspected infection + SOFA score)</li>
      <li>Construction of the final dataset</li>
    </ul>
  </li>
</ul>

<p style="font-size:14px;">
It reconstruct queries from the AI_Clinician repo:
<a href="https://github.com/matthieukomorowski/AI_Clinician">
https://github.com/matthieukomorowski/AI_Clinician
</a>
</p>

<p style="font-size:14px;">
© Dr Matthieu Komorowski, Imperial College London 2019.
</p>

<p style="font-size:13px; color: #555;">
This code is distributed in the hope that it will be useful, but <b>WITHOUT ANY WARRANTY</b>.
</p>

<h2 style="font-size:24px; font-weight:600;">
<b>Extraction of Sub-tables</b>
</h2>

<h3 style="font-size:16px; font-weight:600;">
<b>'Culture' Items</b>
</h3>

<p style="font-size:14px;">
These correspond to blood/urine/CSF/sputum cultures etc.
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.culture AS
SELECT
  subject_id,
  hadm_id,
  icustay_id,
  UNIX_SECONDS(TIMESTAMP(charttime)) AS charttime,
  itemid
FROM physionet-data.mimiciii_clinical.chartevents
WHERE itemid IN (
  6035,3333,938,941,942,4855,6043,2929,225401,225437,225444,225451,
  225454,225814,225816,225817,225818,225722,225723,225724,225725,
  225726,225727,225728,225729,225730,225731,225732,225733,227726,
  70006,70011,70012,70013,70014,70016,70024,70037,70041,225734,
  225735,225736,225768,70055,70057,70060,70063,70075,70083,226131,
  80220
)
ORDER BY subject_id, hadm_id, charttime;

<h3 style="font-size:16px; font-weight:600;">
<b>Microbiologyevents</b>
</h3>

<p style="font-size:14px;">
Extracts date & time of all microbiologyevents (regardless of whether they are positive or negative).
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.microbio AS
SELECT
  subject_id,
  hadm_id,
  UNIX_SECONDS(TIMESTAMP(charttime)) AS charttime,
  UNIX_SECONDS(TIMESTAMP(chartdate)) AS chartdate
FROM physionet-data.mimiciii_clinical.microbiologyevents;

<h3 style="font-size:16px; font-weight:600;">
<b>Antibiotics Administration</b>
</h3>

<p style="font-size:14px;">
Used to find the 'Onset time' of Sepsis infection according to Sepsis-3 definition.
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.abx AS
SELECT
  hadm_id,
  icustay_id,
  UNIX_SECONDS(TIMESTAMP(startdate)) AS startdate,
  UNIX_SECONDS(TIMESTAMP(enddate)) AS enddate
FROM physionet-data.mimiciii_clinical.prescriptions
WHERE gsn IN (
  '002542','002543','007371','008873','008877','008879','008880','008935','008941','008942','008943','008944',
  '008983','008984','008990','008991','008992','008995','008996','008998','009043','009046','009065','009066',
  '009136','009137','009162','009164','009165','009171','009182','009189','009213','009214','009218','009219',
  '009221','009226','009227','009235','009242','009263','009273','009284','009298','009299','009310','009322',
  '009323','009326','009327','009339','009346','009351','009354','009362','009394','009395','009396','009509',
  '009510','009511','009544','009585','009591','009592','009630','013023','013645','013723','013724','013725',
  '014182','014500','015979','016368','016373','016408','016931','016932','016949','018636','018637','018766',
  '019283','021187','021205','021735','021871','023372','023989','024095','024194','024668','025080','026721',
  '027252','027465','027470','029325','029927','029928','037042','039551','039806','040819','041798','043350',
  '043879','044143','045131','045132','046771','047797','048077','048262','048266','048292','049835','050442',
  '050443','051932','052050','060365','066295','067471'
)
ORDER BY hadm_id, icustay_id;

<h3 style="font-size:16px; font-weight:600;">
<b>Demographics</b>
</h3>

<p style="font-size:14px;">
Utilized 'mimiciii_derived' from MIMIC-III BigQuery to obtain the Elixhauser Score.  
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.demog AS
SELECT
    ad.subject_id,
    ad.hadm_id,
    i.icustay_id,

    UNIX_SECONDS(TIMESTAMP(ad.admittime)) AS admittime,
    UNIX_SECONDS(TIMESTAMP(ad.dischtime)) AS dischtime,

    ROW_NUMBER() OVER (PARTITION BY ad.subject_id ORDER BY i.intime ASC) AS adm_order,

    CASE
        WHEN i.first_careunit='NICU'  THEN 5
        WHEN i.first_careunit='SICU'  THEN 2
        WHEN i.first_careunit='CSRU'  THEN 4
        WHEN i.first_careunit='CCU'   THEN 6
        WHEN i.first_careunit='MICU'  THEN 1
        WHEN i.first_careunit='TSICU' THEN 3
    END AS unit,

    UNIX_SECONDS(TIMESTAMP(i.intime)) AS intime,
    UNIX_SECONDS(TIMESTAMP(i.outtime)) AS outtime,
    i.los,

    -- age in years, capped at 90
    CASE
        WHEN TIMESTAMP_DIFF(i.intime, p.dob, DAY) / 365.242 > 89
        THEN 90
        ELSE FLOOR(TIMESTAMP_DIFF(i.intime, p.dob, DAY) / 365.242)
    END AS age,

    UNIX_SECONDS(TIMESTAMP(p.dob)) AS dob,
    UNIX_SECONDS(TIMESTAMP(p.dod)) AS dod,

    p.expire_flag,

    CASE WHEN p.gender='M' THEN 1 WHEN p.gender='F' THEN 2 END AS gender,

    -- died in hosp if recorded DOD is within 24h of discharge
    CAST(TIMESTAMP_DIFF(p.dod, ad.dischtime, SECOND) <= 24*3600 AS INT64) AS morta_hosp,

    -- died within 90 days of ICU admission
    CAST(TIMESTAMP_DIFF(p.dod, i.intime, SECOND) <= 90*24*3600 AS INT64) AS morta_90,

    -- Elixhauser comorbidity sum
    (congestive_heart_failure + cardiac_arrhythmias + valvular_disease +
     pulmonary_circulation + peripheral_vascular + hypertension + paralysis +
     other_neurological + chronic_pulmonary + diabetes_uncomplicated +
     diabetes_complicated + hypothyroidism + renal_failure + liver_disease +
     peptic_ulcer + aids + lymphoma + metastatic_cancer + solid_tumor +
     rheumatoid_arthritis + coagulopathy + obesity + weight_loss +
     fluid_electrolyte + blood_loss_anemia + deficiency_anemias +
     alcohol_abuse + drug_abuse + psychoses + depression) AS elixhauser

FROM physionet-data.mimiciii_clinical.admissions ad
JOIN physionet-data.mimiciii_clinical.icustays i ON ad.hadm_id = i.hadm_id
JOIN physionet-data.mimiciii_clinical.patients p ON p.subject_id = i.subject_id
JOIN physionet-data.mimiciii_derived.elixhauser_quan elix ON elix.hadm_id = ad.hadm_id

ORDER BY subject_id ASC, intime ASC;

<h3 style="font-size:16px; font-weight:600;">
<b>Vital Signs (from 'chartevents')</b>
</h3>

<p style="font-size:14px;">
Divided into 10 chunks of 10000 items for speed.  
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.ce90100 AS
WITH numbered AS (
  SELECT DISTINCT
      icustay_id,
      UNIX_SECONDS(TIMESTAMP(charttime)) AS charttime,
      itemid,
      CASE
          WHEN value = 'None' THEN CAST(0 AS FLOAT64)
          WHEN value = 'Ventilator' THEN CAST(1 AS FLOAT64)
          WHEN value = 'Cannula' THEN CAST(2 AS FLOAT64)
          WHEN value = 'Nasal Cannula' THEN CAST(2 AS FLOAT64)
          WHEN value = 'Face Tent' THEN CAST(3 AS FLOAT64)
          WHEN value = 'Aerosol-Cool' THEN CAST(4 AS FLOAT64)
          WHEN value = 'Trach Mask' THEN CAST(5 AS FLOAT64)
          WHEN value = 'Hi Flow Neb' THEN CAST(6 AS FLOAT64)
          WHEN value = 'Non-Rebreather' THEN CAST(7 AS FLOAT64)
          WHEN value = '' THEN CAST(8 AS FLOAT64)
          WHEN value = 'Venti Mask' THEN CAST(9 AS FLOAT64)
          WHEN value = 'Medium Conc Mask' THEN CAST(10 AS FLOAT64)
          ELSE CAST(valuenum AS FLOAT64)
      END AS valuenum,
      ROW_NUMBER() OVER (ORDER BY charttime) AS rn,
      COUNT(*) OVER() AS total_rows
  FROM physionet-data.mimiciii_clinical.chartevents
  WHERE value IS NOT NULL
    AND itemid IN (
        467, 470,471,223834,227287,194,224691,226707,226730,581,580,
        224639,226512,198,228096,211,220045,220179,225309,6701,6,227243,
        224167,51,455,220181,220052,225312,224322,6702,443,52,456,8368,
        8441,225310,8555,8440,220210,3337,224422,618,3603,615,220277,646,
        834,3655,223762,223761,678,220074,113,492,491,8448,116,1372,1366,
        228368,228177,626,223835,3420,160,727,190,220339,506,505,224700,
        224686,224684,684,224421,224687,450,448,445,224697,444,224695,535,
        224696,543,3083,2566,654,3050,681,2311
    )
)
SELECT icustay_id, charttime, itemid, valuenum
FROM numbered
WHERE rn BETWEEN FLOOR(total_rows/10)*9 + 1 AND FLOOR(total_rows/10)*11
ORDER BY charttime;

<h3 style="font-size:16px; font-weight:600;">
<b>Laboratory Results (from 'chartevents')</b>
</h3>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.labs_ce AS
SELECT
    icustay_id,
    UNIX_SECONDS(TIMESTAMP(charttime)) AS charttime,
    itemid,
    valuenum
FROM physionet-data.mimiciii_clinical.chartevents
WHERE valuenum IS NOT NULL
  AND icustay_id IS NOT NULL
  AND itemid IN (
      829, 1535, 227442, 227464, 4195, 3726, 3792, 837, 220645, 4194, 3725, 3803,
      226534, 1536, 4195, 3726, 788, 220602, 1523, 4193, 3724, 226536, 3747, 225664,
      807, 811, 1529, 220621, 226537, 3744, 781, 1162, 225624, 3737, 791, 1525, 220615,
      3750, 821, 1532, 220635, 786, 225625, 1522, 3746, 816, 225667, 3766, 777, 787,
      770, 3801, 769, 3802, 1538, 848, 225690, 803, 1527, 225651, 3807, 1539, 849,
      772, 1521, 227456, 3727, 227429, 851, 227444, 814, 220228, 813, 220545, 3761,
      226540, 4197, 3799, 1127, 1542, 220546, 4200, 3834, 828, 227457, 3789, 825,
      1533, 227466, 3796, 824, 1286, 1671, 1520, 768, 220507, 815, 1530, 227467, 780,
      1126, 3839, 4753, 779, 490, 3785, 3838, 3837, 778, 3784, 3836, 3835, 776, 224828,
      3736, 4196, 3740, 74, 225668, 1531, 227443, 1817, 228640, 823, 227686
  )
ORDER BY icustay_id, charttime, itemid;

<h3 style="font-size:16px; font-weight:600;">
<b>Laboratory Results (from 'labevents')</b>
</h3>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.labs_le AS
WITH xx AS (
  SELECT DISTINCT
      subject_id,
      hadm_id,
      icustay_id,
      intime,
      outtime
  FROM physionet-data.mimiciii_clinical.icustays
)
SELECT
    xx.icustay_id,
    UNIX_SECONDS(TIMESTAMP(f.charttime)) AS timestp,
    f.itemid,
    f.valuenum
FROM xx
INNER JOIN physionet-data.mimiciii_clinical.labevents AS f
    ON f.hadm_id = xx.hadm_id
   AND f.charttime >= TIMESTAMP_SUB(xx.intime, INTERVAL 1 DAY)
   AND f.charttime <= TIMESTAMP_ADD(xx.outtime, INTERVAL 1 DAY)
WHERE f.itemid IN (
    50971,50822,50824,50806,50931,51081,50885,51003,51222,50810,51301,50983,50902,50809,51006,
    50912,50960,50893,50808,50804,50878,50861,51464,50883,50976,50862,51002,50889,50811,
    51221,51279,51300,51265,51275,51274,51237,50820,50821,50818,50802,50813,50882,50803
)
AND f.valuenum IS NOT NULL
ORDER BY f.hadm_id, timestp, f.itemid;

<h3 style="font-size:16px; font-weight:600;">
<b>Real-time Urine Output</b>
</h3>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.uo AS
SELECT
    icustay_id,
    UNIX_SECONDS(TIMESTAMP(charttime)) AS charttime,
    itemid,
    value
FROM physionet-data.mimiciii_clinical.outputevents
WHERE icustay_id IS NOT NULL
  AND value IS NOT NULL
  AND itemid IN (
      40055,43175,40069,40094,40715,40473,40085,40057,40056,
      40405,40428,40096,40651,226559,226560,227510,226561,227489,
      226584,226563,226564,226565,226557,226558
  )
ORDER BY icustay_id, charttime, itemid;

<h3 style="font-size:16px; font-weight:600;">
<b>Pre-admission Urine Output</b>
</h3>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.preadm_uo AS
SELECT
    oe.icustay_id,
    UNIX_SECONDS(TIMESTAMP(oe.charttime)) AS charttime,
    oe.itemid,
    oe.value,
    TIMESTAMP_DIFF(ic.intime, oe.charttime, MINUTE) AS datediff_minutes
FROM physionet-data.mimiciii_clinical.outputevents AS oe
JOIN physionet-data.mimiciii_clinical.icustays AS ic
  ON oe.icustay_id = ic.icustay_id
WHERE oe.itemid IN (40060, 226633)
ORDER BY oe.icustay_id, charttime, oe.itemid;

<h3 style="font-size:16px; font-weight:600;">
<b>Real-time Fluids Input (from 'metavision')</b>
</h3>

<p style="font-size:14px;">
Notices:

*   Records with no rate = STAT
*   Records with rate = INFUSION
*   Fluids are corrected for tonicity
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.fluid_mv AS
WITH t1 AS (
    SELECT
        icustay_id,
        UNIX_SECONDS(TIMESTAMP(starttime)) AS starttime,
        UNIX_SECONDS(TIMESTAMP(endtime)) AS endtime,
        itemid,
        amount,
        rate,
        CASE
            WHEN itemid IN (30176,30315) THEN amount * 0.25
            WHEN itemid IN (30161) THEN amount * 0.3
            WHEN itemid IN (30020,30015,225823,30321,30186,30211,30353,42742,42244,225159) THEN amount * 0.5
            WHEN itemid IN (227531) THEN amount * 2.75
            WHEN itemid IN (30143,225161) THEN amount * 3
            WHEN itemid IN (30009,220862) THEN amount * 5
            WHEN itemid IN (30030,220995,227533) THEN amount * 6.66
            WHEN itemid IN (228341) THEN amount * 8
            ELSE amount
        END AS tev
    FROM physionet-data.mimiciii_clinical.inputevents_mv
    WHERE icustay_id IS NOT NULL
      AND amount IS NOT NULL
      AND itemid IN (225158,225943,226089,225168,225828,225823,220862,220970,220864,
                     225159,220995,225170,225825,227533,225161,227531,225171,225827,
                     225941,225823,225825,225941,225825,228341,225827,30018,30021,30015,
                     30296,30020,30066,30001,30030,30060,30005,30321,30006,30061,30009,
                     30179,30190,30143,30160,30008,30168,30186,30211,30353,30159,30007,
                     30185,30063,30094,30352,30014,30011,30210,46493,45399,46516,40850,
                     30176,30161,30381,30315,42742,30180,46087,41491,30004,42698,42244)
)

SELECT
    icustay_id,
    starttime,
    endtime,
    itemid,
    ROUND(CAST(amount AS NUMERIC),3) AS amount,
    ROUND(CAST(rate AS NUMERIC),3) AS rate,
    ROUND(CAST(tev AS NUMERIC),3) AS tev
FROM t1
ORDER BY icustay_id, starttime, itemid;

<h3 style="font-size:16px; font-weight:600;">
<b>Real-time fluids input (from 'carevue')</b>
</h3>

<p style="font-size:14px;">
Notices:

*   In 'carevue', all records are considered STAT doses
*   Records with rate = INFUSION
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.fluid_cv AS
WITH t1 AS (
    SELECT
        icustay_id,
        UNIX_SECONDS(TIMESTAMP(charttime)) AS charttime,
        itemid,
        amount,
        CASE
            WHEN itemid IN (30176,30315) THEN amount * 0.25
            WHEN itemid IN (30161) THEN amount * 0.3
            WHEN itemid IN (30020,30321,30015,225823,30186,30211,30353,42742,42244,225159) THEN amount * 0.5
            WHEN itemid IN (227531) THEN amount * 2.75
            WHEN itemid IN (30143,225161) THEN amount * 3
            WHEN itemid IN (30009,220862) THEN amount * 5
            WHEN itemid IN (30030,220995,227533) THEN amount * 6.66
            WHEN itemid IN (228341) THEN amount * 8
            ELSE amount
        END AS tev
    FROM physionet-data.mimiciii_clinical.inputevents_cv
    WHERE amount IS NOT NULL
      AND itemid IN (225158,225943,226089,225168,225828,225823,220862,220970,220864,
                     225159,220995,225170,225825,227533,225161,227531,225171,225827,
                     225941,225823,225825,225941,225825,228341,225827,30018,30021,30015,
                     30296,30020,30066,30001,30030,30060,30005,30321,30006,30061,30009,
                     30179,30190,30143,30160,30008,30168,30186,30211,30353,30159,30007,
                     30185,30063,30094,30352,30014,30011,30210,46493,45399,46516,40850,
                     30176,30161,30381,30315,42742,30180,46087,41491,30004,42698,42244)
)

SELECT
    icustay_id,
    charttime,
    itemid,
    ROUND(CAST(amount AS NUMERIC),3) AS amount,
    ROUND(CAST(tev AS NUMERIC),3) AS tev
FROM t1
ORDER BY icustay_id, charttime, itemid

<h3 style="font-size:16px; font-weight:600;">
<b>Pre-admission Fluids Input</b>
</h3>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.preadm_fluid AS

WITH mv AS (
    SELECT
        ie.icustay_id,
        SUM(ie.amount) AS sum_mv
    FROM physionet-data.mimiciii_clinical.inputevents_mv ie
    JOIN physionet-data.mimiciii_clinical.d_items ci
      ON ie.itemid = ci.itemid
    WHERE ie.itemid IN (30054,30055,30101,30102,30103,30104,30105,30108,
                        226361,226363,226364,226365,226367,226368,226369,
                        226370,226371,226372,226375,226376,227070,227071,227072)
    GROUP BY icustay_id
),

cv AS (
    SELECT
        ie.icustay_id,
        SUM(ie.amount) AS sum_cv
    FROM physionet-data.mimiciii_clinical.inputevents_cv ie
    JOIN physionet-data.mimiciii_clinical.d_items ci
      ON ie.itemid = ci.itemid
    WHERE ie.itemid IN (30054,30055,30101,30102,30103,30104,30105,30108,
                        226361,226363,226364,226365,226367,226368,226369,
                        226370,226371,226372,226375,226376,227070,227071,227072)
    GROUP BY icustay_id
)

SELECT
    ic.icustay_id,
    CASE
        WHEN mv.sum_mv IS NOT NULL THEN mv.sum_mv
        WHEN cv.sum_cv IS NOT NULL THEN cv.sum_cv
        ELSE NULL
    END AS inputpreadm
FROM physionet-data.mimiciii_clinical.icustays ic
LEFT JOIN mv
    ON ic.icustay_id = mv.icustay_id
LEFT JOIN cv
    ON ic.icustay_id = cv.icustay_id
ORDER BY icustay_id;

<h3 style="font-size:16px; font-weight:600;">
<b>Vasopressors (from 'metavision')</b>
</h3>

<p style="font-size:14px;">
Notices:

*   Drugs converted in noradrenaline (NEE)-equivalent
*   Body weight assumed 80 kg when missing
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.vaso_mv AS

SELECT
    icustay_id,
    itemid,
    UNIX_SECONDS(TIMESTAMP(starttime)) AS starttime,
    UNIX_SECONDS(TIMESTAMP(endtime)) AS endtime,
    CASE
        WHEN itemid IN (30120,221906,30047) AND rateuom='mcg/kg/min' THEN ROUND(CAST(rate AS FLOAT64),3) -- norad
        WHEN itemid IN (30120,221906,30047) AND rateuom='mcg/min' THEN ROUND(CAST(rate/80 AS FLOAT64),3) -- norad
        WHEN itemid IN (30119,221289) AND rateuom='mcg/kg/min' THEN ROUND(CAST(rate AS FLOAT64),3) -- epi
        WHEN itemid IN (30119,221289) AND rateuom='mcg/min' THEN ROUND(CAST(rate/80 AS FLOAT64),3) -- epi
        WHEN itemid IN (30051,222315) AND rate > 0.2 THEN ROUND(CAST(rate*5/60 AS FLOAT64),3) -- vasopressin, U/h
        WHEN itemid IN (30051,222315) AND rateuom='units/min' THEN ROUND(CAST(rate*5 AS FLOAT64),3) -- vasopressin
        WHEN itemid IN (30051,222315) AND rateuom='units/hour' THEN ROUND(CAST(rate*5/60 AS FLOAT64),3) -- vasopressin
        WHEN itemid IN (30128,221749,30127) AND rateuom='mcg/kg/min' THEN ROUND(CAST(rate*0.45 AS FLOAT64),3) -- phenyl
        WHEN itemid IN (30128,221749,30127) AND rateuom='mcg/min' THEN ROUND(CAST(rate*0.45/80 AS FLOAT64),3) -- phenyl
        WHEN itemid IN (221662,30043,30307) AND rateuom='mcg/kg/min' THEN ROUND(CAST(rate*0.01 AS FLOAT64),3) -- dopa
        WHEN itemid IN (221662,30043,30307) AND rateuom='mcg/min' THEN ROUND(CAST(rate*0.01/80 AS FLOAT64),3) -- dopa
        ELSE NULL
    END AS rate_std
FROM physionet-data.mimiciii_clinical.inputevents_mv
WHERE itemid IN (30128,30120,30051,221749,221906,30119,30047,30127,221289,222315,221662,30043,30307)
  AND rate IS NOT NULL
  AND statusdescription <> 'Rewritten'
ORDER BY icustay_id, itemid, starttime;

<h3 style="font-size:16px; font-weight:600;">
<b>Vasopressors (from 'carevue')</b>
</h3>

<p style="font-size:14px;">
Notices:

*   Drugs converted in noradrenaline (NEE)-equivalent
*   Body weight assumed 80 kg when missing
</p>

In [ ]:
CREATE OR REPLACE TABLE skripsi-fatas-mimic3.mimic3.vaso_cv AS

SELECT
    icustay_id,
    itemid,
    UNIX_SECONDS(TIMESTAMP(charttime)) AS charttime,
    CASE
        WHEN itemid IN (30120,221906,30047) AND rateuom='mcgkgmin' THEN ROUND(CAST(rate AS FLOAT64),3) -- norad
        WHEN itemid IN (30120,221906,30047) AND rateuom='mcgmin' THEN ROUND(CAST(rate/80 AS FLOAT64),3) -- norad
        WHEN itemid IN (30119,221289) AND rateuom='mcgkgmin' THEN ROUND(CAST(rate AS FLOAT64),3) -- epi
        WHEN itemid IN (30119,221289) AND rateuom='mcgmin' THEN ROUND(CAST(rate/80 AS FLOAT64),3) -- epi
        WHEN itemid IN (30051,222315) AND rate > 0.2 THEN ROUND(CAST(rate*5/60 AS FLOAT64),3) -- vasopressin, U/h
        WHEN itemid IN (30051,222315) AND rateuom='Umin' AND rate <= 0.2 THEN ROUND(CAST(rate*5 AS FLOAT64),3) -- vasopressin
        WHEN itemid IN (30051,222315) AND rateuom='Uhr' THEN ROUND(CAST(rate*5/60 AS FLOAT64),3) -- vasopressin
        WHEN itemid IN (30128,221749,30127) AND rateuom='mcgkgmin' THEN ROUND(CAST(rate*0.45 AS FLOAT64),3) -- phenyl
        WHEN itemid IN (30128,221749,30127) AND rateuom='mcgmin' THEN ROUND(CAST(rate*0.45/80 AS FLOAT64),3) -- phenyl
        WHEN itemid IN (221662,30043,30307) AND rateuom='mcgkgmin' THEN ROUND(CAST(rate*0.01 AS FLOAT64),3) -- dopa
        WHEN itemid IN (221662,30043,30307) AND rateuom='mcgmin' THEN ROUND(CAST(rate*0.01/80 AS FLOAT64),3) -- dopa
        ELSE NULL
    END AS rate_std
FROM physionet-data.mimiciii_clinical.inputevents_cv
WHERE itemid IN (30128,30120,30051,221749,221906,30119,30047,30127,221289,222315,221662,30043,30307)
  AND rate IS NOT NULL
ORDER BY icustay_id, itemid, charttime;

<h3 style="font-size:16px; font-weight:600;">
<b>Mechanical Ventilation</b>
</h3>

In [ ]:
CREATE OR REPLACE TABLE `skripsi-fatas-mimic3.mimic3.mechvent` AS
SELECT
  icustay_id,
  UNIX_SECONDS(TIMESTAMP(charttime)) AS charttime,

  -- Mechanical ventilation flag
  MAX(
    CASE
      WHEN itemid IS NULL OR value IS NULL THEN 0
      WHEN itemid = 720 AND value != 'Other/Remarks' THEN 1
      WHEN itemid = 467 AND value = 'Ventilator' THEN 1
      WHEN itemid IN (
        445, 448, 449, 450, 1340, 1486, 1600, 224687,
        639, 654, 681, 682, 683, 684, 224685, 224684, 224686,
        218, 436, 535, 444, 459, 224697, 224695, 224696, 224746, 224747,
        221, 1, 1211, 1655, 2000, 226873, 224738, 224419, 224750, 227187,
        543,
        5865, 5866, 224707, 224709, 224705, 224706,
        60, 437, 505, 506, 686, 220339, 224700,
        3459,
        501, 502, 503, 224702,
        223, 667, 668, 669, 670, 671, 672,
        157, 158, 1852, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 8382, 227809, 227810,
        224701
      ) THEN 1
      ELSE 0
    END
  ) AS MechVent,

  -- Extubated
  MAX(
    CASE
      WHEN itemid IS NULL OR value IS NULL THEN 0
      WHEN itemid = 640 AND value IN ('Extubated', 'Self Extubation') THEN 1
      ELSE 0
    END
  ) AS Extubated,

  -- Self-extubated
  MAX(
    CASE
      WHEN itemid IS NULL OR value IS NULL THEN 0
      WHEN itemid = 640 AND value = 'Self Extubation' THEN 1
      ELSE 0
    END
  ) AS SelfExtubated

FROM `physionet-data.mimiciii_clinical.chartevents`
WHERE value IS NOT NULL
  AND itemid IN (
    640,
    720,
    467,
    445, 448, 449, 450, 1340, 1486, 1600, 224687,
    639, 654, 681, 682, 683, 684, 224685, 224684, 224686,
    218, 436, 535, 444, 459, 224697, 224695, 224696, 224746, 224747,
    221, 1, 1211, 1655, 2000, 226873, 224738, 224419, 224750, 227187,
    543,
    5865, 5866, 224707, 224709, 224705, 224706,
    60, 437, 505, 506, 686, 220339, 224700,
    3459,
    501, 502, 503, 224702,
    223, 667, 668, 669, 670, 671, 672,
    157, 158, 1852, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 8382, 227809, 227810,
    224701
  )
GROUP BY icustay_id, charttime
ORDER BY icustay_id, charttime;